In [ ]:
import os
from typing import Annotated, TypedDict
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.graph import StateGraph, START, END
from langgraph.agents import create_agent, ToolNode
from langgraph.checkpoint.memory import MemorySaver
from langgraph_supervisor import create_supervisor, create_handoff_tool
from langchain_ollama import ChatOllama  # Ensure async-compatible client
from langchain_core.tools import tool
from dotenv import load_dotenv

load_dotenv()

# --- 1. Set API Keys ---
# No Gemini API key needed; using local Ollama
ollama_host = "http://localhost:11434"  # Default Ollama host; adjust if different

# --- 2. Define Tools ---
@tool
def get_store_info(query: str) -> str:
    """Provide store information based on the query."""
    if "hour" in query.lower() or "open" in query.lower():
        return "The store hours for Chama boutique are 9 AM to 6 PM daily."
    elif "location" in query.lower():
        return "The location for Chama boutique is 123 Fashion St, Nairobi."
    return "Please clarify (e.g., hours or location)."

@tool
def process_order(query: str) -> str:
    """Handle order-related requests."""
    if "place" in query.lower() or "buy" in query.lower():
        return "Order placed successfully for item #123."
    elif "status" in query.lower():
        return "Order #123 is in processing."
    return "Please clarify (e.g., place order or check status)."

@tool
def get_product_info(query: str) -> str:
    """Provide product information based on the query."""
    if "price" in query.lower():
        return "The price of Brown Leather Loafer is $50."
    elif "stock" in query.lower():
        return "Brown Leather Loafer is in stock."
    return "Please clarify (e.g., price or stock)."

# --- 3. Define Agents ---
llm = ChatOllama(
    model="llama3.1:8b",
    base_url="http://192.168.5.116:11434",
    temperature=0,
    # Ensure async support by setting up the client correctly
    streaming=False  # Disable streaming to simplify async handling
)

customer_support = create_react_agent(
    llm,
    tools=[get_store_info],
    name="customer_support",
    prompt=SystemMessage(content="Assist with Chama boutique store inquiries (e.g., hours, location).")
)

order_manager = create_react_agent(
    llm,
    tools=[process_order],
    name="order_manager",
    prompt=SystemMessage(content="Handle Chama boutique order requests (e.g., place, status).")
)

product_query = create_react_agent(
    llm,
    tools=[get_product_info],
    name="product_query",
    prompt=SystemMessage(content="Provide Chama boutique product details (e.g., price, stock).")
)

agents = [customer_support, order_manager, product_query]  # Only include sub-agents

# --- 4. Define State ---
from langgraph.prebuilt.chat_agent_executor import AgentState
AgentState = AgentState  # Use prebuilt AgentState for compatibility

# --- 5. Create Supervisor ---
supervisor_graph = create_supervisor(
    agents=agents,
    model=llm,
    prompt=SystemMessage(content="You are the Chama boutique supervisor. Classify the latest user query's intent as 'query' (for product details like price or stock), 'order' (for order requests like placing or status), 'support' (for store inquiries like hours or location), or 'other' (for unrelated or unclear requests). Route to the appropriate agent (product_query, order_manager, customer_support) based on the intent. For 'other' intents, respond directly with 'I don’t know that'. After delegation, summarize the sub-agent's response and present it to the user."),
    state_schema=AgentState,
    supervisor_name="chama_supervisor"
)

# --- 6. Compile Graph ---
graph = supervisor_graph.compile(checkpointer=MemorySaver())


